In [1]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
#from multiprocessing import Pool, Process
import os
import time


In [10]:
def get_all_page(urls, pattern, url_base):

    if pattern in urls:
        return(urls)
    
    html = requests.get(pattern)
    soup = BeautifulSoup(html.content, "html.parser")
    last_page = soup.find("a", {"title" : "Dernière page"})["page"]

    iter = soup.find_all("div", {"class" : re.compile("m3-pagination print_hidden|print_hidden pagination")})
    for item in iter:
        lst = re.findall("url.*?;", item.find_next_sibling().text)
        nw_url = ""
        if len(lst) > 0:
            for item in lst:
                if re.search("querystring", item):
                    continue
                else:
                    item = re.sub("url =|url \+=|url", "", item)
                    item = item.replace(";", "").replace("\"", "").strip()
                    item = item.split(" + ")
                for part in item:
                    nw_url += part
            match = re.compile("\$\(this\)\.attr\(page\)")
            for i in range(1, int(last_page)+1):
                url = urljoin(url_base, re.sub(match,str(i), nw_url))
                if url not in urls:
                    urls.append(url)
                    
        else:
            for i in range(1, int(last_page)+1):
                urls.append(urljoin(url_base, re.sub("page=.", f"page={i}", pattern)))
    
    return(urls)

In [11]:
def getAllUrl(url, url_base, urls, EXPORT_FILE, DATA_FOLDER, page):

    soup = BeautifulSoup(page.content, "html.parser")
    for anchor in soup.find_all("a"):
        if "href" in anchor.attrs:
            if anchor["href"] == "JavaScript:void(0);" and anchor["class"] == "lien-page":
                try:
                    urls.extend(get_all_page(urls), url, url_base)
                except:
                    continue
            absolute_url = urljoin(url_base, anchor["href"])
            if "download" not in absolute_url and absolute_url not in urls and anchor["href"] != "JavaScript:void(0);":
                if re.search(url_base, absolute_url):
                    urls.append(absolute_url)
    # Export des pages html ?
                    with open(DATA_FOLDER+EXPORT_FILE, "a", encoding="utf-8") as f:
                        f.write(absolute_url + '\n')

    return(urls)

In [12]:
def read_urls(EXPORT_FILE):
    with open(EXPORT_FILE, "r") as f:
        urls = f.read().split("\n") 
    print(len(urls))
    return(urls)

In [14]:

EXPORT_FILE = "M3urls3.txt"
HTML_FILES = "../data/HTMLS.txt"
HTML_FOLDER = "../urls/"
DATA_FOLDER = "../data/"

if __name__ == "__main__":
    
    clean_urls = []
    verify = ""
    STATUS = input( "What do you want to do ? [REBOOT][UPDATE][...] :\n")
    if STATUS == "REBOOT":
        verify = input("REBOOT entraînera la reconstitution complète de la base de données. Êtes-vous sûr de vouloir continuer ? [Y/n]:\n")
            
    
    url_base = 'https://www.millenaire3.com'
    errors = []

    if STATUS == "REBOOT" and verify != "n":
        print("REBOOT")
        urls = get_all_page(urls = [], 
                    pattern="https://www.millenaire3.com/m3_search?page=1&sort_by=0&SearchText=&modeRecherche=1&titre=&auteurId=&auteurText=&100%25=&100%25_new_value=true&date_debut=01%2F01%2F1900&date_debut_mobile=&date_fin=31%2F12%2F2025&date_fin_mobile=&identifiers=&rubriques=",
                    url_base = url_base)
        with open(DATA_FOLDER+EXPORT_FILE, "w", encoding="utf-8") as f:
            for url in urls:
                f.write(url+'\n')
    
    # elif EXPORT_FILE in os.listdir(DATA_FOLDER) and STATUS != "UPDATE":
    #     print("LOAD")
    #     urls = read_urls(DATA_FOLDER+EXPORT_FILE)
        
    else:
        print("UPDATE")
        urls = get_all_page(urls = read_urls(DATA_FOLDER+EXPORT_FILE), 
                    pattern="https://www.millenaire3.com/m3_search?page=1&sort_by=0&SearchText=&modeRecherche=1&titre=&auteurId=&auteurText=&100%25=&100%25_new_value=true&date_debut=01%2F01%2F1900&date_debut_mobile=&date_fin=31%2F12%2F2025&date_fin_mobile=&identifiers=&rubriques=",
                    url_base = url_base)
        with open(DATA_FOLDER+EXPORT_FILE, "w", encoding="utf-8") as f:
            for url in urls:
                f.write(url+'\n')
        
    if verify != "n":
        files = os.listdir(HTML_FOLDER)
        i = 0 #len(files) ? # On peut changer la valeur de départ
        while i <= len(urls)-1:
            URL = urls[i].replace("/", "_").replace(".", "_").replace(":", "_").replace("?", "_").replace("=", "_").replace("&", "_").replace("%", "_").replace("-", "_").replace("httpswwwmillenaire3com", "_").replace("_", "")
            if f"{URL}.html" in files and STATUS != "REBOOT":
                pass
            elif re.search("/partage_email/|/generated_pdf/", urls[i]):
                #print(re.search("/partage_email/|/generated_pdf/", urls[i]))
                errors.append(urls[i])
                print(f"\r{i} / {len(urls)}, errors", end = "")
                pass
            else:
                print(f"\r{i} / {len(urls)}: {urls[i]}", end = "")
                try:
                    page = requests.get(urls[i])
                    if page.status_code == 200:
                        urls = getAllUrl(urls[i], url_base, urls, EXPORT_FILE, DATA_FOLDER, page)
                        with open(f"{HTML_FOLDER}{URL}.html", "w", encoding="utf-8") as f:
                            clean_urls.append(urls[i])
                            f.write(page.text)
                    else:
                        print(f"\r{urls[i]} ", page.status_code, end = "")
                        errors.append(urls[i])
                    time.sleep(0.5)
                except:
                    print(f"Wrong url : {urls[i]}")
            i += 1
            # except:
            #     print("------------ CONNECTION ABORTED ------------")
            #     time.sleep(3)
            #     print("------------ RELAUNCHING ------------")

UPDATE
11404
11351 / 11404: Wrong url : millenaire3.com/dossiers/veille-m3/veille-m3-fermeture-annoncee-d-un-monde-en-ruines-quelle-industrie-a-l-heure-de-l-anthropoceneminants-des-benefices-du-vegetal-sur-la-santeimentaires-actuelles49tropoles-declare-ne-pas-se-nourrir-a-sa-faim
11352 / 11404: Wrong url : 
11355 / 11412: Wrong url : millenaire3.com/dossiers/2023/liens-et-lieux-du-travail-quelles-tendances-a-l-aeuvre-depuis-la-pandemie
11402 / 11536: Wrong url : millenaire3.com/dossiers/2023/ville-permeable-une-nouvelle-approche-mais-au-service-de-quels-enjeuxtidien-des-habitants/participation-electorale-des-qpv-que-nous-disent-les-resultats-des-scrutins-nationauxr-les-mobilitesdans-le-coliving
11403 / 11536: Wrong url : 
11624 / 11625, errors//www.millenaire3.com/dossiers/2023/la-consommation-de-bio-a-la-croisee-des-chemins/amelie-arnould-carrefour-le-marche-bio-a-encore-du-potentiel-et-il-pourra-rebondirionnement-est-largement-oriente-pour-produire-du-capital-blitesobilitea-cyclomobi

In [33]:
#### CHECK 
with open("../data/M3urls3.txt", "r", encoding="utf-8") as f:
    urls2 = f.read().split('\n')

6231 11352 6145
httpswwwmillenaire3com touslesdossiers .html  --  https://www.millenaire3.com
httpswwwmillenaire3com#1cop21.html  --  https://www.millenaire3.com#%5Bi%5Dcop2
httpswwwmillenaire3com#1grdv.html  --  https://www.millenaire3.com#.UTy7Xsd6oFc.facebook
httpswwwmillenaire3com#2grdv.html  --  https://www.millenaire3.com#1cop21
httpswwwmillenaire3com#5Bi5Dcop2.html  --  https://www.millenaire3.com#1grdv
httpswwwmillenaire3com#Ancre1AP.html  --  https://www.millenaire3.com#2grdv
httpswwwmillenaire3com#Ancre2AP.html  --  https://www.millenaire3.com#Ancre1_AP
httpswwwmillenaire3com#UTy7Xsd6oFcfacebook.html  --  https://www.millenaire3.com#Ancre2_AP
httpswwwmillenaire3com#a1.html  --  https://www.millenaire3.com#_a1
httpswwwmillenaire3com#a2.html  --  https://www.millenaire3.com#_a2
httpswwwmillenaire3com#a3.html  --  https://www.millenaire3.com#_a3
httpswwwmillenaire3com#a4.html  --  https://www.millenaire3.com#_a4
httpswwwmillenaire3com#a5.html  --  https://www.millenaire3.com#_ft